# Projektaufgabe - Roundtrip-Path

Implementieren Sie einen Roundtrip-Path Planer

1. Gegeben sind  
    1. Startposition  
    2. Mehrere Endpositionen
    3. Das Interface des Roundtrip-Path-Planners soll sich *nicht* von den anderen Bahnplanern unterscheiden.
    4. Der zu verwendete Bahnplanungsalgorithmus soll wählbar sein und geeignet übertragen werden können.
    5. Ausgabe: Ein kollisionsfreier Pfad, der von der gegebenen Startposition alle Endpositionen genau einmal erreicht und wieder zur Startposition zurückführt. Dabei soll der Gesamtpfad möglichst kurz sein. Koordinieren Sie den Pfad geeignet (z.B. farblich), so dass sich erkennen lässt, was Start-Punkt, kollisionsfreier Zwischenpfad und Zielpunkte sind. \
    *Anmerkung: NetworkX hat prinzipiell die Möglichkeit auf einem gegebenen Graph dieses Problem zu lösen, sogar auf verschiedene Arten und Weisen. Sie können dies nutzen*

In [ ]:
import networkx as nx
from IPPerfMonitor import IPPerfMonitor

In [ ]:
import math
from typing import List, Any
import numpy as np
from numpy.typing import NDArray


class Roundtrip_path_planner:


    def __init__(self, path_planner):
        assert hasattr(path_planner, "planPath"), "path_planner must have a method called 'planPath'"
        self.graph = nx.Graph() # graph to store all paths between start and goal nodes
        self.tsg_graph = nx.Graph() # graph to store the Traveling Salesman Graph (TSG) containing only start and goal nodes. Each edge contains the actual path between the nodes and the respective summed weight.
        self.path_planner = path_planner
        self._collisionChecker = path_planner._collisionChecker if hasattr(path_planner, "_collisionChecker") else None


    def _compute_path_length(self, graph: nx.Graph, path: List[Any]) -> float:
        """
        Computes the total length of a path in the graph.
        Args:
            graph (nx.Graph): The graph containing the path.
            path (List[Any]): A list of nodes representing the path.
        returns:
            float: The total distance of the path. Returns 0.0 if the path is empty or has less than 2 nodes.
        """
        if not path or len(path) < 2:
            return 0.0
        total_distance = sum(
            math.dist(graph.nodes[u]['pos'], graph.nodes[v]['pos'])
            for u, v in zip(path[:-1], path[1:])
        )
        return total_distance


    

    @IPPerfMonitor
    def planPath(self, startList: List[List[Any]], goalList: List[List[Any]], config) -> List[Any]:
        """
        Plans a roundtrip path that visits all start and goal nodes.
        Args:
            startList (array): start position in planning space. E.g. [[1,2]]
            goalList (array) : goal position in planning space. E.g. [[3,4]]
            config (dict): dictionary with the needed information about the configuration options

        Returns:
            List[List[Any]]: A list representing the roundtrip path visiting all goals and returning to the start.
        """
        start_goal_coordinates = startList + goalList # e.g. [[1,2], [3,4]]
        # add start and target nodes to the graph
        for i, coordinates in enumerate(start_goal_coordinates):
            self.tsg_graph.add_node(i, pos=coordinates)
        # get added target nodes
        target_nodes = self.tsg_graph.nodes
        # Copy the TSG graph to the main graph
        self.graph = self.tsg_graph.copy()
        
        
        # for each target nodes pair compute the path between them
        for i, start_node in enumerate(target_nodes):
            for j, goal_node in enumerate(target_nodes):
                if j <= i:
                    continue
                # plan path from start_node to goal_node
                path: List[Any] = self.path_planner.planPath([self.tsg_graph.nodes[start_node]['pos']], [self.tsg_graph.nodes[goal_node]['pos']], config)
                # Relabel nodes so composition does not merge different nodes (different pos)
                mapping = {node: (start_node if node == "start" else goal_node if node == "goal" else f'{start_node}_{goal_node}_{node}') for node in self.path_planner.graph.nodes}
                self.path_planner.graph = nx.relabel_nodes(self.path_planner.graph, mapping, copy=False)
                path = [mapping[node] for node in path]
                # Merge self.graph and planner.graph
                self.graph = nx.compose(self.path_planner.graph, self.graph)
                # Update TSG graph
                self.tsg_graph.add_edge(start_node, 
                                        goal_node, 
                                        distance=self._compute_path_length(self.graph, path), 
                                        path=path)
                

        
        # Use the Traveling Salesman Problem approximation to find a roundtrip path
        tsg_solution = np.asarray(nx.algorithms.approximation.traveling_salesman_problem(
            self.tsg_graph,
            weight='distance',
            nodes=target_nodes,
            cycle=True
        ))

        if len(tsg_solution) < 2:
            return []

        # Reconstruct the path from the TSG solution
        roundtrip_path = [0]
        for i in range(len(tsg_solution) - 1):
            path = self.tsg_graph[tsg_solution[i]][tsg_solution[i + 1]]['path']
            if tsg_solution[i] > tsg_solution[i + 1]:
                path = list(reversed(path))
            roundtrip_path.extend(path[1:])
        
        return roundtrip_path


2. Evaluieren Sie ihr Konzept mit BasicPRM, LazyPRM, VisibilityPRM anhand von mindestens 6 Benchmarkumgebungen (3 * 2-DoF Punktroboter, 3 * Planarroboter). Die Szenen der Benchmarkaufgaben für die 2-DoF Punktroboter sollen unterschiedlich aussehen und unterschiedlich schwer sein. Die Szene der Benchmarkaufgaben für die Planarroboter können gleich sein und lediglich die Anzahl der Freiheitsgrade und Start, Ziel und Zwischenpunkte verändert werden.
    1. Betrachten Sie z.B. Anzahl der Kollisionsberechnungen, Planungszeit, Roadmapgröße, Länge Lösungspfad und Stellen Sie die Ergebnisse graphisch dar und diskutieren Sie diese.
    2. Für die 2-DoF Punkt- und Planarroboter stellen Sie sowohl die Vorgehensweise des Roundtrip-Planers (welche Bahnen werden wann geplant, was ist das finale Ergebnis) als auch das Endergebnis als Animation (s. IP-10-0-PlanarManipulator.ipnyb) dar.

In [ ]:
import matplotlib.pyplot as plt
from IPBenchmark import Benchmark
from shapely.geometry import Point, Polygon, LineString

from IPEnvironment import CollisionChecker
from IPVISBasicPRM import basicPRMVisualize
import IPTestSuite as ts

from IPVisibilityPRM import VisPRM

In [ ]:
import traceback


visConfig = dict()
visConfig["ntry"] = 100

for benchmark in ts.benchList:
    fig_local = plt.figure(figsize=(10,10))
    ax = fig_local.add_subplot(1,1,1)
    planner = VisPRM(benchmark.collisionChecker)
    roadmap_planner = Roundtrip_path_planner(planner)
    solution = roadmap_planner.planPath(benchmark.startList, benchmark.goalList, visConfig)
    title = benchmark.name
    if solution == []:
        title += " (No path found!)"
    title += "\n Assumed complexity level " + str(benchmark.level)
    ax.set_title(title)
    print("solution:", solution)
    # TODO: Solution consists currently of a list of coordinates. For visualization, we need the planner to contain a graph and the solution to contain node names.
    basicPRMVisualize(roadmap_planner, solution, ax=ax, nodeSize=50)


In [ ]:
import traceback


visConfig = dict()
visConfig["ntry"] = 100

for benchmark in ts.benchList:
    try:
        fig_local = plt.figure(figsize=(10,10))
        ax = fig_local.add_subplot(1,1,1)
        planner = VisPRM(benchmark.collisionChecker)
        roadmap_planner = Roundtrip_path_planner(planner)
        solution = roadmap_planner.planPath(benchmark.startList, benchmark.goalList, visConfig)
        title = benchmark.name
        if solution == []:
            title += " (No path found!)"
        title += "\n Assumed complexity level " + str(benchmark.level)
        ax.set_title(title)
        print("solution:", solution)
        # TODO: Solution consists currently of a list of coordinates. For visualization, we need the planner to contain a graph and the solution to contain node names.
        basicPRMVisualize(roadmap_planner, solution, ax=ax, nodeSize=50)
    except Exception as e:
        print("ERROR in benchmark:", benchmark.name)
        print("Exception:", e)
        traceback.print_exc()

3. In einem weiteren Schritt entwickeln Sie eine spezielle Variante auf Basis des VisibilityPRM. Wie Sie wissen, ist dieser für Multi-Query Anfragen vorteilhaft nutzbar. Nutzen Sie dies für das Roundtrip-Konzept. Hier müssen Sie eventuell die Art wie Sie den Visibility-PRM aufrufen geschickt verändern. Führen Sie ebenfalls die Evaluation an den Benchmarkumgebungen durch und vergleichen Sie die Ergebnisse mit den Versionen aus 2.) 

Erkläutern Sie bitte zudem im Endbericht (mind. 1 Seite):
1. Wie funktioniert der Roundtrip-Planer, den Sie verwenden im Detail und warum haben Sie diesen gewählt.
2. Wie können Sie die Bewegungsbahnen optimieren/Glätten? Erläutern SIe kurz eine mögliche Vorgehensweise.

Anmerkung: Bitte checken Sie das Notebook "IP-X-0-Benchmarking-concept.ipnyb" und "IP-X-1-Automated_PlanerTest.ipnyb" für Profiling und Statistiken.